## Обобщен Knapsack проблем

### Проект на Владимир Михов
ФН - 61941

#### Проблемът
Обобщеният Knapsack проблем се дефинира по следния начин:
Дадени са:
* N на брой раници, всяка от които може да носи определено тегло
* M на брой предмети, всеки от които има тегло и цена, като теглото и цената зависят от раницата, в която се намира предметът

Целта на задачата е да се намери най-доброто разпределение на предметите по раниците, така че цената да е максимална.

#### Алгоритъмът
За решение на проблема се използва <b>симулирано охлаждане</b>. Алгоритъмът работи по следния начин:
* Започваме с празни раници и някаква начална температура
* Търсим близко състояние, като взимаме случаен предмет и го поставяме в случайна раница с вероятност 90% или просто го оставяме
* Ако цената е по-голяма в новото състояние, то приемаме новото състояние. Ако цената е по-малка - приемаме новото състояние с вероятност P
* Повтаряме краен брой пъти

$$ P = \exp \left( \frac{\Delta E}{kT}\right) $$

$$ \Delta E = f(x_{proposed}) - f(x_{current})$$

In [1]:
import numpy as np
import os

def get_data(dataset):
    dataset_dir = 'datasets'

    base_dir = os.path.join(dataset_dir, dataset)

    capacities_file = os.path.join(base_dir, 'c.txt')
    weights_file = os.path.join(base_dir, 'w.txt')
    profits_file = os.path.join(base_dir, 'p.txt')
    solution_file = os.path.join(base_dir, 's.txt')

    with open(capacities_file) as f:
        capacities = np.array(f.readlines(), dtype=int)

    with open(weights_file) as f:
        weights = np.array(
            list(map(lambda l: np.array(l.split(), dtype=int),f.readlines()))
        )

    with open(profits_file) as f:
        profits = np.array(
            list(map(lambda l: np.array(l.split(), dtype=int),f.readlines()))
        )

    if os.path.exists(solution_file):
        with open(solution_file) as f:
            solution = np.array(f.readline().split(), dtype=int)
    else:
        solution = None

    return capacities, weights, profits, solution


In [44]:
from math import exp

capacities, weights, profits, solution = get_data('3')
knapsacks = weights.shape[0]
objects = weights.shape[1]

def is_valid(state):
    knapsack_weights = np.multiply(state, weights).sum(axis=1)
    return np.all(knapsack_weights <= capacities)

def profit(state):
    return np.sum(np.multiply(profits,state))

def acceptance(profit_new, profit_current, T, k=0.1):
    return exp((profit_new - profit_current)/k/T)

def cooling(T, T_start, step, total_steps):
    return T_start * (1 - step/total_steps)
    
def neighbour(state):
    state_copy = state.copy()
    random_object = np.random.randint(objects)

    state_copy[:,random_object] = 0
    if np.random.rand() < 0.9:
        random_knapsack = np.random.randint(knapsacks)
        state_copy[random_knapsack, random_object] = 1
    
    return state_copy

def generate_random_state():
    while True:
        object_indices = np.random.randint(0, knapsacks, objects)
        state = np.zeros(weights.shape, dtype=int)

        state[object_indices, np.arange(objects)] = 1

        if is_valid(state):
            return state


class SimulatedAnnealingKnapsack:
    def __init__(
            self,
            neighbour=neighbour,
            profit=profit,
            acceptance=acceptance,
            cooling=cooling,
            T_start=100.0,
            total_steps=5000):
        self.state_current = np.zeros(weights.shape)#_generate_random_state()
        self.profit_current = 0

        self.state_best = None
        self.profit_best = 0

        self.neighbour = neighbour
        self.profit = profit
        self.acceptance = acceptance
        self.cooling = cooling
        self.T_start = T_start
        self.total_steps = total_steps
    
    def iteration(self, T):
        state_new = self.neighbour(self.state_current)
        if not is_valid(state_new):
            return

        profit_new = self.profit(state_new)

        dE = self.profit_current - profit_new

        if dE < 0:
            accept = True
        else:
            accept = np.random.rand() < self.acceptance(profit_new, self.profit_current, T)
        
        if accept:
            self.state_current = state_new
            self.profit_current = profit_new
            if profit_new > self.profit_best:
                self.profit_best = profit_new
                self.state_best = state_new
    
    def anneal(self):
        T = self.T_start
        for step in range(self.total_steps+1):
            if step % (self.total_steps/10) == 0:
                print(f'Step - {step}, Temp - {T:.1f}, Profit - {self.profit_current}')
            self.iteration(T)
            T = self.cooling(T, self.T_start, step, self.total_steps)

In [62]:
optimizer = SimulatedAnnealingKnapsack(total_steps=10000)

optimizer.anneal()

print(f'Best positions: {np.argmax(optimizer.state_best, axis=0)+1}')
print(f'Best profit: {optimizer.profit_best}\n')

if solution is not None:
    desired_state = np.zeros(weights.shape, dtype=int)
    desired_state[solution-1, np.arange(objects)] = 1
    desired_profit = profit(desired_state)

    print(f'Desired positions: {solution}')
    print(f'Desired profit: {desired_profit:.1f}')

Step - 0, Temp - 100.0, Profit - 0
Step - 1000, Temp - 90.0, Profit - 232.0
Step - 2000, Temp - 80.0, Profit - 226.0
Step - 3000, Temp - 70.0, Profit - 226.0
Step - 4000, Temp - 60.0, Profit - 218.0
Step - 5000, Temp - 50.0, Profit - 218.0
Step - 6000, Temp - 40.0, Profit - 232.0
Step - 7000, Temp - 30.0, Profit - 232.0
Step - 8000, Temp - 20.0, Profit - 232.0
Step - 9000, Temp - 10.0, Profit - 232.0
Step - 10000, Temp - 0.0, Profit - 232.0
Best positions: [3 3 1 1 2 2 1 2]
Best profit: 232.0

Desired positions: [3 3 1 1 2 2 1 2]
Desired profit: 232.0
